In [ ]:
import requests
import json
import os
import pandas as pd
import geopandas as gpd

In [ ]:
def run_overpass_query(ql_query):
        """Run an overpass query"""

        overpass_url = "http://overpass-api.de/api/interpreter"
        response = requests.get(overpass_url, params={"data": ql_query})
        if response.content:
            try:
                data = response.json()
            except:
                data = {"error": str(response)}
        else:
            data = {
                "Warning": "Received an empty response from Overpass API. Tell the user."
            }
        data_str = json.dumps(data)

        return data_str


query_bus_stops_berlin = """[out:json][timeout:100];
area[name="Dresden"]->.searchArea;
(
nwr(area.searchArea)["highway"="bus_stop"];
);
out;"""


SAVE_DIRECTORY = "../../data/raw"
filename = "bus_stops_dresden.json"
berlin_bus_stops_path = os.path.join(SAVE_DIRECTORY, filename)

if not os.path.exists(berlin_bus_stops_path):
    data = run_overpass_query(query_bus_stops_berlin)
    with open(berlin_bus_stops_path, "w") as file:
        json.dump(json.loads(data), file)

In [ ]:
def load_osm_result(path):
    with open(path, "r") as f:
        osm_response = json.load(f)

    data = []
    for element in osm_response["elements"]:
        if element["type"] == "node" and "lat" in element and "lon" in element:
            properties = element.get("tags", {})
            properties.update({"id": element["id"], "lat": element["lat"], "lon": element["lon"]})
            data.append(properties)

    # Create DataFrame
    df = pd.DataFrame(data)

    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
    gdf.set_crs(epsg=4326, inplace=True)# Set the CRS to WGS84 (EPSG:4326)
    return gdf

berlin_bus_stops_gdf = load_osm_result(berlin_bus_stops_path)

berlin_bus_stops_gdf.set_crs(epsg=4326, inplace=True)